<a href="https://colab.research.google.com/github/AndrewHamal/got/blob/main/image_to_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install tesserocr
!pip install opencv-python
!pip install Pillow
!pip install pytesseract
!pip install git+https://github.com/casics/nostril.git
!git clone https://github.com/tesseract-ocr/tessdata.git
!git clone https://github.com/rrenaud/Gibberish-Detector
!pip install Flask


In [ ]:
!pip install gibberish-detector
!git clone https://github.com/rrenaud/Gibberish-Detector.git

fatal: destination path 'Gibberish-Detector' already exists and is not an empty directory.


In [ ]:
!gibberish-detector train Gibberish-Detector/big.txt > big.model

In [ ]:

import tesserocr
from nostril import nonsense
import cv2
from PIL import Image, ImageEnhance
from google.colab.patches import cv2_imshow
import pytesseract
import random
import string
import re
import numpy as np
from gibberish_detector import detector
import requests
from io import BytesIO
import urllib.request
import nltk
from flask import Flask

def imageToText(url):
  # gibberish_detector set model
  Detector = detector.create_from_model('big.model')
  response = requests.get(url)

  # load image from url in PIL image and convert to RGB/jpg format
  img = Image.open(requests.get(url, stream=True).raw)
  rgb_im = img.convert('RGB')

  # Decrease saturation of image
  converter = ImageEnhance.Color(rgb_im)
  pil_img = converter.enhance(0)

  # react image from url in open cv
  req = urllib.request.urlopen(url)
  arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
  final = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)

  #initialize tesserocr api
  api = tesserocr.PyTessBaseAPI(path='tessdata')
  api.SetImage(pil_img)

  # differentiate all the chat boxes based of text line in array
  boxes = api.GetComponentImages(tesserocr.RIL.TEXTLINE, True)
  height, width, c = final.shape

  inc = int(0.01*width)
  list_all = []

  # cv2_imshow(final)

  for i, (im,box,_,_) in enumerate(boxes):
    x,y,w,h = box['x'],box['y'],box['w'],box['h']
    crop_img = final[max(0, y-inc):min(y+h+inc,height), max(0, x-inc):min(x+w+inc, width)]
    api.SetRectangle(box['x'], box['y'], box['w'], box['h'])

    #  cv2_imshow(crop_img)
    #  cv2.waitKey(0)

    extractedInformation = pytesseract.image_to_string(crop_img)
    stringAll = extractedInformation.replace('|', 'I').replace('\n\x0c', ' ');

    try:
      if not Detector.is_gibberish(stringAll):
        if (x < 200 and stringAll != '\x0c' and not re.search("(Type a message Send|Snapchat|Ne Sent|Delivered|Message\n\n)", stringAll) and not re.search("((1[0-2]|0?[1-9]):([0-5][0-9]))", stringAll)):
          list_all.append({"body": stringAll, "sender": 'Partner'})
        elif (x > 199 and stringAll != '\x0c' and not re.search("Type a message Send|Snapchat|Ne Sent|Delivered|Message\n\n", stringAll) and not re.search("((1[0-2]|0?[1-9]):([0-5][0-9]))", stringAll)):
          list_all.append({"body": stringAll, "sender": 'Me'})
    except: ''

  return list_all

url = "https://cp-kyle.sfo3.cdn.digitaloceanspaces.com/instant-responses/XZN9oUX7XPlufI2QeslKK3Pk8JQsVhjYnQZD86MZ.png"
print(imageToText(url))

# app = Flask(__name__)

# @app.route('/<url>')
# def hello_world(url):
#   return 'Hello World'


# if __name__ == '__main__':
#   app.run()


[{'body': 'christina ', 'sender': 'Me'}, {'body': 'I live in Munich, originally from Egypt ', 'sender': 'Me'}, {'body': 'Has anyone taught you some English ', 'sender': 'Me'}, {'body': 'sweat words yet;)? ', 'sender': 'Me'}, {'body': 'Don’t be shy ', 'sender': 'Me'}, {'body': 'What are you doing in Munich? ', 'sender': 'Partner'}, {'body': '@) I Know some English but not very well ', 'sender': 'Partner'}, {'body': "I’m running a business with family so I'm ", 'sender': 'Me'}, {'body': 'taking care of things here..also my mom ', 'sender': 'Me'}, {'body': 'told me my dream troublemaker girl ', 'sender': 'Me'}, {'body': 'named Christina was here ', 'sender': 'Me'}, {'body': 'I can teach you some more English in ', 'sender': 'Me'}, {'body': 'exchange for a few cute sentences in ', 'sender': 'Me'}, {'body': 'Russian ', 'sender': 'Me'}, {'body': 'You are cute 2 ', 'sender': 'Partner'}, {'body': '@) Nice, I think we can do this @ ', 'sender': 'Partner'}]
